In [119]:
#API package import and key input

from googleapiclient.discovery import build

api_key = input()

    
youtube = build('youtube','v3',developerKey = api_key)

YT API KEY GOES HERE


In [120]:
#video ID input
ytid = input()




VIDEO ID GOES HERE


In [125]:
#this will store paginated toplevel comment json dictionaries
dictlist = []

#this makes the initial request for toplevel comments, if there is more than 100 comments it will have a nextPageToken
dict_output = youtube.commentThreads().list(part="snippet,replies", videoId=ytid, maxResults = 100).execute()
dictlist.append(dict_output)
npt = dict_output.get('nextPageToken')

#this keeps requesting new jsons with new nextPageTokens as long as the last json requested had one
while npt:
    dict_output = youtube.commentThreads().list(part="snippet,replies", videoId=ytid, pageToken = npt, maxResults = 100).execute()
    dictlist.append(dict_output)
    npt = dict_output.get('nextPageToken')

#this will store the actual output
comments = []  

#we loop through all output jsons and enter every thread (thread = toplevel comment)
for dict in dictlist:
    for thread in dict['items']:
        onecomment = []
        #this takes the actual text of the comment and puts it into its own little list
        #reads the number of replies to the comment too
        commenttext = thread['snippet']['topLevelComment']['snippet']['textDisplay']
        replycount = thread['snippet']['totalReplyCount']
        onecomment.append(commenttext)
        
        #if the toplevel comment has replies, we'll need to request them
        if replycount>0:
            targettoplevelid = thread['snippet']['topLevelComment']['id']
            replydictlist = []
            #same logic as with toplevel comments
            #request initial 100 replies, if the json came back with a nextPageToken request again
            #keep requesting until json comes back without a nextPageToken
            reply_dict_output = youtube.comments().list(part="snippet", parentId = targettoplevelid, maxResults = 100).execute()
            replydictlist.append(reply_dict_output)
            replynpt = reply_dict_output.get('nextPageToken')
            while replynpt:
                reply_dict_output = youtube.comments().list(part="snippet", parentId = targettoplevelid, pageToken = replynpt, maxResults = 100).execute()
                replydictlist.append(reply_dict_output)
                replynpt = reply_dict_output.get('nextPageToken')
            #loops through reply jsons, takes the actual text of the reply and puts it into yet another list
            #I guess I really like lists
            parsedreplies = []
            for replydict in replydictlist:
                for comment in replydict['items']:
                    parsedreplies.append(comment['snippet']['textOriginal'])
            onecomment.append(parsedreplies)
            #this appends to the output with the comment
            #the output it structured like this
            #[['top level with no replies'],['top level with replies', ['reply1','reply2']], ...]
  

        comments.append(onecomment)
        
    

In [126]:
#testing cell - run to check if all comments were pulled
#number should be equal or within 1-2 of the number on the YT video page
#YT counters are occasionally inaccurate

i = 0
for element in comments:
    if len(element) == 1:
        i += 1
 
j = 0
for element in comments:
    if len(element) > 1:
        j += 1

l = 0
for element in comments:
    if len(element) > 1:
        l += len(element[1])
        
i+j+l

9008